# OPN 10: Task a asynchronní programování
---

Obsah:
- Task
- async a await


## Task

Pomocí třídy `Task` můžeme spouštět typicky asynchronně jednu operaci (metodu) bez návratové hodnoty. 

V následujícím příkladu spustíme provádění metody Metoda pomocí statické metody `Task.Run`, tato metoda nám vrátí proměnnou `task`, krerá popisuje již spuštěnou operaci běžící typicky v jiném vlákně. Kompilátor se totiž může rozhodnout, zda se vyplatí operaci pouštět v jiném vlákně nebo ji stačí provést sychronně ve stejném vlákně. 

Příkaz `task.Wait();` pozastaví běh hlavního vlákna programu.

In [ ]:
void DlouhoBeziciMetoda()
{
    for (int i = 0; i < 10; i++)
    {
        Console.WriteLine(i);
        System.Threading.Thread.Sleep(1000);
    }
}

Task task = Task.Run(DlouhoBeziciMetoda);
Console.WriteLine("Task spusten");
task.Wait();
Console.WriteLine("Konec programu");

Příkaz `await task;` nepozastaví běh programu, ale jen přeruší provádění metody `Main` a mezití běží ostatní operace a po dokončení metody `Metoda` bude v provádění metody `Main` pokračovat. 

In [ ]:
void DlouhoBeziciMetoda()
{
    for (int i = 0; i < 10; i++)
    {
        Console.WriteLine(i);
        System.Threading.Thread.Sleep(400);
    }
}

Task task = Task.Run(DlouhoBeziciMetoda);
Console.WriteLine("Task spusten");
await task;
Console.WriteLine("Konec programu");


In [1]:
#r "nuget: Avalonia.Desktop"

using Avalonia;
using Avalonia.Controls;
using Avalonia.Controls.ApplicationLifetimes;
using Avalonia.Layout;

Installed Packages Avalonia.Desktop, 0.10.19

Loading extensions from `C:\Users\erik\.nuget\packages\skiasharp\2.88.1-preview.108\interactive-extensions\dotnet\SkiaSharp.DotNet.Interactive.dll`

In [2]:
if(Application.Current is null)
{
    AppBuilder.Configure<Application>().UsePlatformDetect().SetupWithLifetime(new ClassicDesktopStyleApplicationLifetime());

    Application.Current.Styles.Add(new Avalonia.Themes.Fluent.FluentTheme(new Uri("avares://MyAssembly")) { Mode = Avalonia.Themes.Fluent.FluentThemeMode.Dark});
}

In [6]:
if(Application.Current.ApplicationLifetime is ClassicDesktopStyleApplicationLifetime desktop)
{
    Window window = new Window() 
    { 
        Title = "Avalonia"
    };

    TextBlock textBlock = new TextBlock() 
    { 
        Text = "Ahoj", 
        HorizontalAlignment = HorizontalAlignment.Center, 
        VerticalAlignment = VerticalAlignment.Center 
    };

    window.Content = textBlock;

    desktop.MainWindow = window;

    desktop.Start(new string[0]);
}